In [ ]:
'''
Citation:

[1] “Pandas,” pandas. [Online]. Available: https://pandas.pydata.org/.
[2] "NumPy." [Online]. Available: https://numpy.org/.
'''

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
O_R_df = pd.read_csv('../Webscrapping/webdata/Opiate-Recovery.csv')
R_df = pd.read_csv('../Webscrapping/webdata/reddit_effects.csv')
T_df = pd.read_csv('../Webscrapping/webdata/twitter_effects.csv')
Comb_R_T_df =  pd.concat([R_df,T_df])
Comb_R_T_df = Comb_R_T_df.reset_index(drop=True)

In [ ]:
SEnts = []
EEnts = []
# Finding out the Substance and effects from the pretrained NER model
import spacy
import pandas as pd
nlp = spacy.load("en_reddit_ner\en_reddit_ner")
for i in range(len(O_R_df['Comments'])):
    doc = nlp(O_R_df['Comments'][i])
    SEnts.append([ent.text for ent in doc.ents if ent.label_=='SUBSTANCE'])
    EEnts.append([ent.text for ent in doc.ents if ent.label_=='EFFECT'])

O_R_df.insert(3,'PSE',EEnts,True)
O_R_df.insert(2,'Components',SEnts,True)
O_R_df['PSE'] = O_R_df['PSE'].apply(lambda y: np.nan if len(y)==0 else ", ".join(y))
O_R_df['Components'] = O_R_df['Components'].apply(lambda y: np.nan if len(y)==0 else ", ".join(y))
O_R_df = O_R_df[O_R_df['Components'].notna()]
O_R_df.sort_values(by=["Components"],inplace = True)
O_R_df = O_R_df.reset_index(drop=True)

In [ ]:
EEnts = []
for i in range(len(Comb_R_T_df['Comments'])):
    doc = nlp(Comb_R_T_df['Comments'][i])
    EEnts.append([ent.text for ent in doc.ents if ent.label_=='EFFECT'])

Comb_R_T_df.insert(3,'PSE',EEnts,True)
Comb_R_T_df['PSE'] = Comb_R_T_df['PSE'].apply(lambda y: np.nan if len(y)==0 else ", ".join(y))
Comb_R_T_df.sort_values(by=["Components"],inplace = True)
Comb_R_T_df = Comb_R_T_df.reset_index(drop=True)

In [ ]:
final =  pd.concat([O_R_df,Comb_R_T_df])
final = final.reset_index(drop=True)
final.to_csv('train.csv')